# Taller de equilibrios de Nash

In [8]:
import numpy as np
import time
from gurobipy import *

In [9]:
class NashEqulibriumSolver:

    def __init__(self, game_step, p1_action_num, p2_action_num, payoff_matrix):
        
        """
        game_step: step number of the game; normal game step is 1, extensive game step is more than 1
        p1_action_num: player 1's action number
        p2_action_num: player 2's action number
        payoff_matrix: the generated payoff matrix of the game
        """

        self.game_step = game_step
        self.action_choice_player1 = p1_action_num
        self.action_choice_player2 = p2_action_num
        self.action_total_num_player1 = self.action_choice_player1 ** game_step
        self.action_total_num_player2 = self.action_choice_player2 ** game_step
        self.payoff_matrix = payoff_matrix
        
        assert self.payoff_matrix.shape[0] == self.action_total_num_player1, "Payoff matrix row number does not match Player 1 pure strategy number"
        assert self.payoff_matrix.shape[1] == self.action_total_num_player2, "Payoff matrix column number does not match Player 2 pure strategy number"
    
    def solve_linear_program_player1(self, verbose=False):

        """
        Solve the NE profile strategy for player 1

        verbose: output result details or not

        Return (filtered_player1_NE, NE_utility), where filtered_player1_NE is player 1's strategy, 
                                                        and NE_utility is its NE profile utility
        """

        start_time = time.time()

        ## setup model
        LP_model = Model()
        LP_model.setParam('OutputFlag', 0)

        ## define variables
        game_value = LP_model.addVar(name='game_value', vtype=GRB.CONTINUOUS) # the NE profile utility, i.e., the value of the zero-sum two-player game
        player1_actions = LP_model.addVars(self.action_total_num_player1, vtype=GRB.CONTINUOUS) # the probabilities for all actions of player 1

        ## add constraints

        # all player 1 actions' probability should be non-negative, and their sum is 1
        LP_model.addConstr(player1_actions.sum() == 1)
        LP_model.addConstrs(player1_actions[i] >= 0 for i in range(self.action_total_num_player1))
       
        # bound player 1's utility by game_value (for each player 1's pure strategy)
        for player2_action_index in range(self.action_total_num_player2):
            lhs = 0
            for player1_action_index in range(self.action_total_num_player1):
                # if the payoff matrix is huge (e.g., converted from extensive game), we can store it with bool utilities to save space
                if type(self.payoff_matrix[player1_action_index, player2_action_index]) is np.bool_:
                    p1_utility = 1 if self.payoff_matrix[player1_action_index, player2_action_index] else -1
                # for simple normal form game, just store all payoff values
                else:
                    p1_utility = self.payoff_matrix[player1_action_index, player2_action_index]
                lhs += p1_utility * player1_actions[player1_action_index]
            LP_model.addConstr(lhs >= game_value)


        ## setup objective func and solve
        obj = game_value
        LP_model.setObjective(obj, GRB.MAXIMIZE)

        LP_model.optimize()

        ## return solved outcomes
        if LP_model.Status == GRB.Status.OPTIMAL:
            player1_NE = LP_model.getAttr('x', player1_actions)
            filtered_player1_NE = {k: v for k,v in player1_NE.items() if v > 0.}
            NE_utility = LP_model.ObjVal
            if verbose:
                print("Player 1 strategy solving time: ", time.time() - start_time)
                print("Nash Equlibrium Profile Utility for Player 1: ", NE_utility)
                print("Nash Equlibrium Player 1 Strategy: ", filtered_player1_NE)
                print("------------------------------------------")
            return (filtered_player1_NE, NE_utility)
        else:
            print(LP_model.Status, GRB.Status.OPTIMAL)
            print("The linear programming for player 1 is infeasible, please double check.")
            return None
    
    def solve_linear_program_player2(self, verbose=False):

        """
        Solve the NE profile strategy for player 2

        verbose: output result details or not

        Return (filtered_player2_NE, NE_utility), where filtered_player2_NE is player 2's strategy, 
                                                        and NE_utility is its NE profile utility
        """

        start_time = time.time()

        ## setup model
        LP_model = Model()
        LP_model.setParam('OutputFlag', 0)

        ## define variables
        game_value = LP_model.addVar(name='game_value', vtype=GRB.CONTINUOUS) # the NE profile utility, i.e., the value of the zero-sum two-player game
        player2_actions = LP_model.addVars(self.action_total_num_player2, vtype=GRB.CONTINUOUS) # the probabilities for all actions of player 2

        ## add constraints

        # all player 2 actions' probability should be non-negative, and their sum is 1
        LP_model.addConstr(player2_actions.sum() == 1)
        LP_model.addConstrs(player2_actions[i] >= 0 for i in range(self.action_total_num_player2))
       
        # bound player 2's utility by game_value (for each player 1's pure strategy)
        for player1_action_index in range(self.action_total_num_player1):
            lhs = 0
            for player2_action_index in range(self.action_total_num_player2):
                # if the payoff matrix is huge (e.g., converted from extensive game), we can store it with bool utilities to save space
                if type(self.payoff_matrix[player1_action_index, player2_action_index]) is np.bool_:
                    p1_utility = 1 if self.payoff_matrix[player1_action_index, player2_action_index] else 1
                # for simple normal form game, just store all payoff values
                else:
                    p1_utility = self.payoff_matrix[player1_action_index, player2_action_index]
                lhs += p1_utility * player2_actions[player2_action_index]
            LP_model.addConstr(lhs <= game_value)


        ## setup objective func and solve
        obj = game_value
        LP_model.setObjective(obj, GRB.MINIMIZE)

        LP_model.optimize()

        ## return solved outcomes
        if LP_model.Status == GRB.Status.OPTIMAL:
            player2_NE = LP_model.getAttr('x', player2_actions)
            filtered_player2_NE = {k: v for k,v in player2_NE.items() if v > 0.}
            NE_utility = LP_model.ObjVal
            if verbose:
                print("Player 2 strategy solving time: ", time.time() - start_time)
                print("Nash Equlibrium Profile Utility for Player 2: ", - NE_utility)
                print("Nash Equlibrium Player 2 Strategy: ", filtered_player2_NE)
                print("------------------------------------------")
            return (filtered_player2_NE, NE_utility)
        else:
            print(LP_model.Status, GRB.Status.OPTIMAL)
            print("The linear programming for player 2 is infeasible, please double check.")
            return None

In [10]:
# solve the game
def solve(game_step, p1_action_choice, p2_action_choice, payoff_matrix, verbose=False):

    # some basic setup
    p1_action_num = len(p1_action_choice)
    p2_action_num = len(p2_action_choice)

    # solving
    solver = NashEqulibriumSolver(game_step, p1_action_num, p2_action_num, payoff_matrix)
    player1_sol = solver.solve_linear_program_player1(verbose=verbose)
    player2_sol = solver.solve_linear_program_player2(verbose=verbose)

    # results
    print("-------------Player 2 strategy-------------")
    if player2_sol is not None:
        player2_strategy = player2_sol[0]
        for k, v in player2_strategy.items():
            action = p2_action_choice[k]
            print(action, v)
    else:
        print('')

    print("-------------Player 1 strategy-------------")
    if player1_sol is not None:
        player1_strategy = player1_sol[0]
        for k, v in player1_strategy.items():
            action = p1_action_choice[k]
            print(action, v)
    else:
        print('')

| Juego                     | Jugadores | Estrategias por jugador | Nº de equilibrios de Nash en estrategias puras | Secuencial | Información perfecta | Suma cero |
|---------------------------|-----------|--------------------------|-----------------------------------------------|------------|----------------------|-----------|
| Batalla de los sexos      | 2         | 2                        | 2                                             | No         | No                   | No        |
| Adivina 2/3 del promedio  | N         | infinitas                | 1                                             | No         | No                   | No        |
| Cara o cruz               | 2         | 2                        | 0                                             | No         | No                   | Sí        |
| Dilema del prisionero     | 2         | 2                        | 1                                             | No         | No                   | No        |
| Juegos de Blotto   | 2         | variable                      | variable                                        | No         | No                   | Sí        |
| Cortando pastel    | N         | infinito                      | variable                                        | Sí         | Sí                   | Sí        |

## Batalla de los sexos

Imagina que un hombre y una mujer esperan encontrarse esta noche, pero tienen que elegir entre dos eventos a los que asistir: una pelea de boxeo y un ballet. El hombre preferiría ir a la pelea de boxeo. La mujer preferiría el ballet. Ambos prefieren ir al mismo evento en lugar de ir a eventos diferentes. Si no pueden comunicarse, ¿a dónde deberían ir?

|                 | Pelea de Boxeo | Ballet |
|-----------------|----------------|--------|
| **Pelea de Boxeo** | 3, 2          | 1, 1   |
| **Ballet**        | 0, 0          | 2, 3   |



In [11]:
payoff_matrix = np.array([
    [[3, 2], [1, 1]],  # Opciones si eligen "Pelea de Boxeo"
    [[0, 0], [2, 3]]   # Opciones si eligen "Ballet"
])

game_step = 1
p1_actions = {0: "Box", 1: "Ballet"}
p2_actions = {0: "Box", 1: "Ballet"}

In [ ]:
# Resolvemos el juego
solve(game_step, p1_actions, p2_actions, payoff_matrix)

## Adivina 2/3 del promedio

Referencia: https://www.researchgate.net/publication/24091873_A_Cognitive_Hierarchy_Model_Games

El juego "Adivina 2/3 del promedio" en teoría de juegos consiste en que los jugadores seleccionan simultáneamente un número real entre 0 y 100, inclusivamente. El ganador del juego es aquel (o aquellos) que eligen un número que esté más cercano a 2/3 del promedio de los números elegidos por todos los jugadores.

Este juego ilustra cómo los jugadores intentan anticipar las elecciones de los demás para ganar. Si todos los jugadores pensaran racionalmente, el proceso llevaría a la eliminación iterativa de estrategias dominadas, lo que resulta en el número 0 como solución de equilibrio de Nash.

Sin embargo, en la práctica, los resultados suelen reflejar diferentes niveles de razonamiento y suposiciones sobre el comportamiento de los demás.

<img src='./imgs/23.png'>

## Monedas coincidentes

El juego de las monedas coincidentes (Matching Pennies) es un juego no cooperativo. Se juega entre dos participantes, Par (Even) e Impar (Odd). Cada jugador tiene una moneda y debe, en secreto, girarla hacia cara o cruz. Luego, ambos jugadores revelan sus elecciones de manera simultánea.

Si las monedas coinciden (ambas caras o ambas cruces), entonces Par gana y se queda con ambas monedas. Si las monedas no coinciden (una cara y la otra cruz), entonces Impar gana y se queda con ambas monedas.

In [6]:
payoff_matrix = np.array([[+1, -1], [-1, +1]])
game_step = 1
p1_actions = {0: "Cara", 1: "Cruz"}
p2_actions = {0: "Cara", 1: "Cruz"}

In [7]:
# Resolvemos el juego
solve(game_step, p1_actions, p2_actions, payoff_matrix)

-------------Player 2 strategy-------------
Cara 0.5
Cruz 0.5
-------------Player 1 strategy-------------
Cara 0.5
Cruz 0.5


Comentario sobre las estrategias: https://en.wikipedia.org/wiki/Matching_pennies

## El dilema del prisionero

Comentario sobre cooperación dada iteración: https://en.wikipedia.org/wiki/Prisoner%27s_dilemma

## Juegos de Blotto

Un juego del Coronel Blotto es un tipo de juego de suma constante para dos personas en el que los jugadores (oficiales) tienen la tarea de distribuir simultáneamente recursos limitados en varios objetos (campos de batalla). En la versión clásica del juego, el jugador que dedica más recursos a un campo de batalla gana ese campo, y la ganancia (o pago) es igual al número total de campos de batalla ganados.

Este juego se usa comúnmente como una metáfora para la competencia electoral, donde dos partidos políticos dedican dinero o recursos para atraer el apoyo de un número fijo de votantes. Cada votante es un "campo de batalla" que puede ser ganado por uno u otro partido. El mismo juego también tiene aplicación en la teoría de subastas, donde los postores deben hacer ofertas simultáneas.

## Cortando pastel


Cortando pastel es un tipo de problema de división justa. 

El problema involucra un recurso heterogéneo, como un pastel con diferentes coberturas, que se asume divisible, es decir, es posible cortar piezas arbitrariamente pequeñas sin destruir su valor. 

El recurso debe ser dividido entre varios socios que tienen diferentes preferencias sobre las diferentes partes del pastel, es decir, algunas personas prefieren las coberturas de chocolate, otras prefieren las cerezas, algunas solo quieren la pieza más grande posible. La división debe ser unánimemente justa: cada persona debe recibir una pieza que se considere una parte justa.